In [0]:
import pandas as pd
import numpy as np
import sklearn
import nltk

In [0]:
train_data = pd.read_csv("train.txt",sep='\t')
train_data.head()
train_data['single'] = train_data['turn1']+" "+train_data['turn2']+" "+train_data['turn3']
def drop_features(features,data):
    data.drop(features,inplace=True,axis=1)
# drop_features(['id','turn1','turn2','turn3'],train_data)
train_data.head()

,id,turn1,turn2,turn3,label,single
0,0,Don't worry I'm girl,hmm how do I know if you are,What's ur name?,others,Don't worry I'm girl hmm how do I know if you...
1,1,When did I?,saw many times i think -_-,No. I never saw you,angry,When did I? saw many times i think -_- No. I n...
2,2,By,by Google Chrome,Where you live,others,By by Google Chrome Where you live
3,3,U r ridiculous,I might be ridiculous but I am telling the truth.,U little disgusting whore,angry,U r ridiculous I might be ridiculous but I am ...
4,4,Just for time pass,wt do u do 4 a living then,Maybe,others,Just for time pass wt do u do 4 a living then ...


In [0]:
from sklearn.model_selection import train_test_split

x_train,x_test, y_train, y_test = train_test_split(train_data["single"],train_data["label"], test_size = 0.2, random_state = 40) 

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)
# print(count_vect.vocabulary_)

In [0]:
x_train_counts = count_vect.fit_transform(x_train)
x_train_tfidf = transformer.fit_transform(x_train_counts)
print(x_train_counts.shape)
print(x_train_tfidf.shape)

(24128, 12275)
(24128, 12275)


In [0]:
x_test_counts = count_vect.transform(x_test)
x_test_tfidf = transformer.transform(x_test_counts)
print(x_test_counts.shape)
print(x_test_tfidf.shape)

(6032, 12275)
(6032, 12275)


In [0]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=50)
model.fit(x_train_tfidf,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [0]:
predictions = model.predict(x_test_tfidf)

In [0]:
from sklearn.metrics import confusion_matrix,f1_score
confusion_matrix(y_test,predictions)

array([[ 833,   20,  183,   46],
       [  26,  513,  304,   32],
       [  86,  124, 2659,  135],
       [  48,   28,  317,  678]], dtype=int64)

In [0]:
f1_score(y_test,predictions,average='macro')

0.7435110295923117

In [0]:
from sklearn.metrics import precision_score,recall_score

In [0]:
precision_score(y_test,predictions, average='macro')

0.7791378338143373

In [0]:
recall_score(y_test,predictions, average='macro')

0.718590668679216

In [0]:
test_data = pd.read_csv("devwithoutlabels.txt",sep='\t')

In [0]:
test_data['single'] = test_data['turn1']+" "+test_data['turn2']+" "+test_data['turn3']
# drop_features(['id','turn1','turn2','turn3'],test_data)
test_data.head()

,id,turn1,turn2,turn3,single
0,0,Then dont ask me,YOURE A GUY NOT AS IF YOU WOULD UNDERSTAND,IM NOT A GUY FUCK OFF,Then dont ask me YOURE A GUY NOT AS IF YOU WOU...
1,1,Mixed things such as??,the things you do.,Have you seen minions??,Mixed things such as?? the things you do. Hav...
2,2,Today I'm very happy,and I'm happy for you ❤,I will be marry,Today I'm very happy and I'm happy for you ❤ I...
3,3,Woah bring me some,left it there oops,Brb,Woah bring me some left it there oops Brb
4,4,it is thooooo,I said soon master.,he is pressuring me,it is thooooo I said soon master. he is pressu...


In [0]:
train_counts = count_vect.fit_transform(train_data['single'])
test_counts = count_vect.transform(test_data['single'])

In [0]:
print(train_counts.shape)
print(test_counts.shape)

(30160, 13968)
(2755, 13968)


In [0]:
train_tfidf = transformer.fit_transform(train_counts)
test_tfidf = transformer.transform(test_counts)

In [0]:
print(train_tfidf.shape)
print(test_tfidf.shape)

(30160, 13968)
(2755, 13968)


In [0]:
model.fit(train_tfidf,train_data['label'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [0]:
predictions = model.predict(test_tfidf)

In [0]:
final_result = pd.DataFrame({'turn1':test_data['turn1'],'turn2':test_data['turn2'],'turn3':test_data['turn3'],'label':predictions})
final_result.index.names=['id']
final_result.to_csv('test.txt',sep='\t')